## Merge dataset from august

- 8월 이후로 수집된 데이터를 모두 합하고 필요하지 않은 columns 삭제
- 활동일을 정규화, 중간 파일을 `data\extracted\merged_route_check_01.csv`, `data\extracted\merged_route_check_02.csv`로 저장
- `merged_route_check_02.csv`파일 수작업으로 데이터 처리 후 환자 정보 없는 데이터는 `merged_route_check_03_01.csv`로, 환자 정보를 가진 데이터는 `merged_route_check_03_02.csv`로 저장

In [5]:
import sys
import os

from os import listdir
from os.path import isfile, join
from pathlib import Path

root = Path(os.getcwd()).parent.parent
sys.path.append(str(root))

In [6]:
import pandas as pd
import numpy as np
import datetime
import math
import re

from datetime import date, timedelta

8월 이후 수집한 데이터 목록 가져오기

In [3]:
'''
dataset_path = join(root, 'data', 'extracted')
dataset_list = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
dataset_list
'''

"\ndataset_path = join(root, 'data', 'extracted')\ndataset_list = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]\ndataset_list\n"

In [4]:
filename_list = ['dobong1.csv', 'dongdaemoon1.csv', 'dongdaemoon2.csv',
                'dongjak1.csv', 'eunpyeong1.csv', 'gangbuk1.csv',
                'gangdong1.csv', 'gangdong2.csv', 'gangnam1.csv',
                'gangnam2.csv', 'gangnam3.csv', 'gangseo1.csv',
                'gangseo2.csv', 'gangseo3.csv', 'gangseo4.csv',
                'gangseo5.csv', 'guro1.csv', 'gwanak1.csv',
                'gwanak2.csv', 'jongno1.csv', 'jung1.csv',
                'jungnang1.csv', 'jungnang2.csv', 'jungnang3.csv',
                'mapo1.csv', 'mapo2.csv', 'nowon1.csv',
                'seocho1.csv', 'seodaemoon1.csv', 'seongbuk1.csv',
                'seongdong1.csv', 'seongdong2.csv', 'songpa1.csv',
                'yeongdeungpo1.csv', 'yongsan1.csv']
file_path = join(root, 'data', 'extracted')

data_list = []
for filename in filename_list:
    data = pd.read_csv(join(file_path, filename))
    data['from'] = re.search(r'[a-z]*(?=\d)', filename).group()
    data_list.append(data)
    
merged_data = pd.concat(data_list, axis=0)
merged_data

NameError: name 'pd' is not defined

In [ ]:
merged_data.columns

column 통일 methods

In [ ]:
def is_nan(row, column):
    if row[column] == np.nan: return True
    else: return False
    
def merge_column(df, target_column, column_list):
    for column in column_list:
        for index, data in df.iterrows():
            if is_nan(data, target_column) and not is_nan(data. column):
                df.loc[index][target_column] = data[column]
    
    new_index = df.columns
    for column in column_list:
        new_index = new_index.drop(column)
        
    df = df[new_index.tolist()]
    return df

확진일 관련 column `comfirmed_date`로 통일

In [ ]:
merged_data = merge_column(merged_data, 'confirmed_date', ['확진일', 'confiemd_date'])

노출일자 관련 column `date`로 통일

In [ ]:
merged_data = merge_column(merged_data, 'date', [' date', '노출일자'])

환자 일련번호 관련 column `patient_id`로 통일(`local_id` 제외)

In [ ]:
merged_data = merge_column(merged_data, 'patient_id', ['id', 'patent_id', 'patient', 'global_id', '확진자'])

감염경로 관련 column `reason`으로 통일

In [ ]:
merged_data = merge_column(merged_data, 'reason', ['감염경로'])

방문지 주소 관련 column `location_address`로 통일(`region` 제외)

In [ ]:
merged_data = merge_column(merged_data, 'location_address', [' location_address', 'exact_address', '주소', 'location'])

방문지 지역 관련 column `location_region`으로 통일

In [ ]:
merged_data = merge_column(merged_data, 'location_region', [' location_region'])

방문지 유형 관련 column `location_type`으로 통일

In [ ]:
merged_data = merge_column(merged_data, 'location_type', ['유형', 'place_type', 'type', 'location_name', ' location_type'])

인적사항 관련 column `personal_info`로 통일

In [ ]:
merged_data = merge_column(merged_data, 'personal_info', ['인적사항', '개인정보'])

소독여부 관련 column `is_infected`로 통일

In [ ]:
merged_data = merge_column(merged_data, 'is_infected', ['disinfection', '소독 여부'])

In [ ]:
merged_data

In [ ]:
merged_data.columns

불필요한 columns 제거

In [ ]:
columns_to_drop = ['접촉', '격리시설', 'is_infected']

new_index = merged_data.columns
for column in columns_to_drop:
    new_index = new_index.drop(column)
    
merged_data = merged_data[new_index.tolist()]
merged_data

중간 저장

path = join(root, 'data', 'extracted', 'merged_route_check_01.csv')
merged_data.to_csv(path, encoding='utf-8-sig', index=False)

중간 불러오기

In [147]:
path = join(root, 'data', 'extracted', 'merged_route_check_01.csv')
merged_data = pd.read_csv(path)

date 조정

In [148]:
def date_into_format(str_date, previous_date_list):
    if not isinstance(str_date, str): return []
    
    # 일반 날짜 형식 정규식
    p_date1 = '(\d+\.)*\d+\.\s*\d+\.*(\([가-힣]\))*\s*' # 8.27(목) 형식
    p_date2 = '\d+\.\s*\d+\.\d+\.*(\([가-힣]\))*\s*' # 20.8.27(목) 형식
    p_date3 = '\d+월\s*\d+일*\s*(\([가-힣]\))*\s*' # 1월 1일(수) 형식
    p_date4 = '\d+\/\d+(\([가-힣]\))*' # 1/1(수) 형식
    p_date5 = '\d+\.*\([가-힣]\)'

    pattern_list = [p_date1, p_date2, p_date3, p_date4]

    # 중복 확인하기 위한 정규식
    dup_date1 = '~\s*(\d+\.)*\d+\.\s*\d+\.*(\([가-힣]\))*' # ~1.1(월) 형식
    dup_date2 = '~\s*\d+\s일*(\([가-힣]\))*' # ~1일(월) 형식
    dup_date3 = '~\s*\d+\/\d+(\([가-힣]\))*' # ~1/1(월) 형식
    
    dup_list = [dup_date1, dup_date2, dup_date3]
    
    # 중복 있는 경우
    elements = []
    for dup_pattern in dup_list:
        end_date = re.search(r'%s' % dup_pattern, str_date)
        if end_date is None: continue
        
        # 끝나는 날짜
        end_date = end_date.group()
        elements = [int(x.group()) for x in re.finditer(r'\d+', end_date)]
        
        break

    # 일반적인 경우
    found_date_list = []
    formed_date_list = []
    for pattern in pattern_list:
        found_date = re.search(r'%s' % pattern, str_date)
        if found_date is None: continue
        
        found_date_list = [x.group() for x in re.finditer(r'%s' % pattern, str_date)]
        for date in found_date_list:
            date_elem_list = [int(x.group()) for x in re.finditer(r'\d+', date)]
            if len(date_elem_list) > 2 and date_elem_list[0] > 12:
                del date_elem_list[0]
            if date_elem_list[0] > 12: continue
            formed_date = datetime.datetime(2020, date_elem_list[0], date_elem_list[1], 0, 0)
            formed_date = formed_date.strftime('%Y-%m-%d')
            formed_date_list.append(formed_date)
            
        break
        
    # 중복 있는 경우 마지막 날부터 중복 날까지 append
    if len(elements) != 0:
        if len(elements) == 1:
            month = int(re.search(r'(?<=-).*(?=-)', formed_date_list[-1]).group())
            day = elements[0]
        elif len(elements) == 2:
            month = elements[0]
            day = elements[1]
        else:
            month = elements[1]
            day = elements[2]
        
        end_date = datetime.datetime(2020, month, day, 0, 0)
        normal_last_day = formed_date_list[-1]
        normal_last_day = datetime.datetime.strptime(normal_last_day, '%Y-%m-%d')
        delta = end_date - normal_last_day
        
        dates_to_append = []
        for i in range(delta.days + 1):
            day = normal_last_day + timedelta(days=i)
            dates_to_append.append(day.strftime('%Y-%m-%d'))
        
        formed_date_list += dates_to_append
        formed_date_list.append(normal_last_day)
        
    # formed_date_list가 비어 있는 경우. 이전 row의 날짜 받는지, 일만 표기되어 있는 것이 아닌지
    if len(formed_date_list) == 0:
        # 이전 row의 날짜 받는지
        if re.search(r'^\s*-\s*$', str_date) is not None:
            return previous_date_list
        if re.search(r'%s' % p_date5, str_date) is not None:
            day_list = [x.group() for x in re.finditer(r'%s' % p_date5, str_date)]
            for i, day in enumerate(day_list):
                day_list[i] = int(re.search(r'\d+', day).group())
                
            if len(previous_date_list) == 0: return formed_date_list
            
            month = int(re.search(r'(?<=-).*(?=-)', previous_date_list[-1]).group())
            for day in day_list:
                new_date = datetime.datetime(2020, month, day, 0, 0)
                new_date = new_date.strftime('%Y-%m-%d')
                formed_date_list.append(new_date)
        
    return formed_date_list

In [149]:
found_date_list = []
new_row_list = []
indices_to_del = []
for index, data in merged_data.iterrows():
    found_date_list = date_into_format(data['date'], found_date_list)
    
    if len(found_date_list) == 0:
        continue
    if len(found_date_list) == 1:
        merged_data.loc[index, 'date'] = found_date_list[0]
    if len(found_date_list) > 1:
        for found_date in found_date_list:
            new_row = merged_data.loc[index]
            new_row['date'] = found_date
            new_row_list.append(new_row)
        
        indices_to_del.append(index)
        
merged_data = merged_data.drop(indices_to_del)

for new_row in new_row_list:
    merged_data = merged_data.append(new_row, ignore_index=True)

merged_data.reset_index(drop=True, inplace=True)

중간 저장

In [150]:
path = join(root, 'data', 'extracted', 'merged_route_check_02.csv')
merged_data.to_csv(path, encoding='utf-8-sig', index=False)